# App to Prevent Gambling Addictions

A medical institute that aims to prevent and treat gambling addictions wants to build a dedicated mobile app to help lottery addicts better estimate their chances of winning. The institute has a team of engineers that will build the app, but they need us to create the logical core of the app and calculate probabilities.

For the first version of the app, they want us to focus on the 6/49 lottery and build functions that enable users to answer questions like:

- What is the probability of winning the big prize with a single ticket?
- What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
- What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

# Defining Functions

Throughout the project, we'll need to calculate repeatedly probabilities and combinations. As a consequence, we'll start by writing two functions that we'll use often:

- A function that calculates factorials; and
- A function that calculates combinations.

In [1]:
# Example of a factorial number: 5! = 5*4*3*2*1
def factorial(n):
    terms = [n]
    c = n
    for i in range(n):
        if c != 1:
            terms.append(c-1)
            c -= 1
        else:
            break
    product = 1
    for term in terms:
        product *= term
    return product

def combinations(n, k):
    combination = factorial(n)/(factorial(n-k) * factorial(k))
    return combination   

# One-Ticket Probability

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket. Furthermore, we'll show the probability value in a friendly way — in a way that people without any probability training are able to understand.

In [2]:
def one_ticket_probability(list_6):
    len_list = len(list_6)
    all_outcomes = combinations(49, len_list)
    win_probability_pct = (1/all_outcomes)*100
    return print('''Your chances to win the big prize with the numbers {} are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(list_6,
                    win_probability_pct, int(all_outcomes)))

print(one_ticket_probability([1,2,3,4,5,6]))

Your chances to win the big prize with the numbers [1, 2, 3, 4, 5, 6] are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.
None


# Canada Lottery Historical Data

For the first version of the app, users should also be able to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

Now, we'll focus on exploring the historical data coming from the Canada 6/49 lottery. The data set can be downloaded [here](https://www.kaggle.com/datascienceai/lottery-dataset).

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
lottery = pd.read_csv('649.csv')
print(lottery.shape)
lottery.head(3)

(3665, 11)


,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,6/12/1982,3,11,12,14,41,43,13
1,649,2,0,6/19/1982,8,33,36,37,39,41,9
2,649,3,0,6/26/1982,1,6,23,24,27,39,34


In [5]:
lottery.tail(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
3662,649,3589,0,6/13/2018,6,22,24,31,32,34,16
3663,649,3590,0,6/16/2018,2,15,21,31,38,49,8
3664,649,3591,0,6/20/2018,14,24,31,35,37,48,17


# Function for Historical Data Check

The engineering team wants us to write a function that prints:
- The number of times the combination selected occurred in the Canada data set; and
- The probability of winning the big prize in the next drawing with that combination.

In [6]:
def extract_numbers(row):
    row = row[4:10]
    row = set(row.values)
    return row

lottery['winning_numbers'] = lottery.apply(extract_numbers, axis=1)
lottery.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER,winning_numbers
0,649,1,0,6/12/1982,3,11,12,14,41,43,13,"{3, 41, 11, 12, 43, 14}"
1,649,2,0,6/19/1982,8,33,36,37,39,41,9,"{33, 36, 37, 39, 8, 41}"
2,649,3,0,6/26/1982,1,6,23,24,27,39,34,"{1, 6, 39, 23, 24, 27}"


In [7]:
def check_historical_occurence(list_6, historic=lottery['winning_numbers']):
    set_numbers = set(list_6)
    series = historic == set_numbers
    counter = series.sum()
    return print('''Number of times the combination {} occurred: {}    Regardless whether this combination occurred or not, your chances to win are the same.
Your chances to win the big prize with these numbers are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.'''.format(list_6,
                    counter))

check_historical_occurence(list_6=[3,11,12,14,41,44])

Number of times the combination [3, 11, 12, 14, 41, 44] occurred: 0    Regardless whether this combination occurred or not, your chances to win are the same.
Your chances to win the big prize with these numbers are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.


# Multi-Ticket Probability

Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning — we're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

- The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
- Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
- The function should print information about the probability of winning the big prize depending on the number of different tickets played.

In [8]:
def multi_ticket_probability(n_tickets):
    all_outcomes = combinations(49, 6)
    probability = (n_tickets/all_outcomes)*100
    if n_tickets == 1:
        print('''Your chances to win the big prize with one ticket are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(probability, int(all_outcomes)))
    
    else:
        combinations_simplified = round(all_outcomes / n_tickets)   
        print('''Your chances to win the big prize with {:,} different tickets are {:.7f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_tickets, probability,
                                                               combinations_simplified))

testing = [1, 10, 100, 10000, 1000000, 6991908, 13983816]
for n in testing:
    multi_ticket_probability(n)
    print('------------------------') # output delimiter

Your chances to win the big prize with one ticket are 0.0000072%.
In other words, you have a 1 in 13,983,816 chances to win.
------------------------
Your chances to win the big prize with 10 different tickets are 0.0000715%.
In other words, you have a 1 in 1,398,382 chances to win.
------------------------
Your chances to win the big prize with 100 different tickets are 0.0007151%.
In other words, you have a 1 in 139,838 chances to win.
------------------------
Your chances to win the big prize with 10,000 different tickets are 0.0715112%.
In other words, you have a 1 in 1,398 chances to win.
------------------------
Your chances to win the big prize with 1,000,000 different tickets are 7.1511238%.
In other words, you have a 1 in 14 chances to win.
------------------------
Your chances to win the big prize with 6,991,908 different tickets are 50.0000000%.
In other words, you have a 1 in 2 chances to win.
------------------------
Your chances to win the big prize with 13,983,816 differ

# Less Winning Numbers

We're going to write one more function to allow the users to calculate probabilities for two, three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket match two, three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having two, three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

- Inside the app, the user inputs:
    - six different numbers from 1 to 49; and
    - an integer between 2 and 5 that represents the number of winning numbers expected
- Our function prints information about the probability of having the inputted number of winning numbers.

In [9]:
def probability_less_6(n_winning_numbers):
    n_combinations_ticket = combinations(6, n_winning_numbers)
    n_combinations_remaining = combinations(43, 6 - n_winning_numbers)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    
    n_combinations_total = combinations(49, 6)    
    probability = successful_outcomes / n_combinations_total
    
    probability_percentage = probability * 100    
    combinations_simplified = round(n_combinations_total/successful_outcomes)    
    print('''Your chances of having {} winning numbers with this ticket are {:.6f}%.
In other words, you have a 1 in {:,} chances to win.'''.format(n_winning_numbers, probability_percentage,
                                                               int(combinations_simplified)))

In [12]:
testing_2 = [2, 3, 4, 5]
for n in testing_2:
    probability_less_6(n)
    print('------------------------') # output delimiter

Your chances of having 2 winning numbers with this ticket are 13.237803%.
In other words, you have a 1 in 8 chances to win.
------------------------
Your chances of having 3 winning numbers with this ticket are 1.765040%.
In other words, you have a 1 in 57 chances to win.
------------------------
Your chances of having 4 winning numbers with this ticket are 0.096862%.
In other words, you have a 1 in 1,032 chances to win.
------------------------
Your chances of having 5 winning numbers with this ticket are 0.001845%.
In other words, you have a 1 in 54,201 chances to win.
------------------------


# Next steps

For the first version of the app, we coded four main functions:

- one_ticket_probability() — calculates the probability of winning the big prize with a single ticket
- check_historical_occurrence() — checks whether a certain combination has occurred in the Canada lottery data set
- multi_ticket_probability() — calculates the probability for any number of of tickets between 1 and 13,983,816
- probability_less_6() — calculates the probability of having two, three, four or five winning numbers exactly

Possible features for a second version of the app include:

- Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "You are 100 times more likely to be the victim of a shark attack than winning the lottery")
- Combining the one_ticket_probability() and check_historical_occurrence() to output information on probability and historical occurrence at the same time
- Create a function similar to probability_less_6() which calculates the probability of having at least two, three, four or five winning numbers. Hint: the number of successful outcomes for having at least four winning numbers is the sum of these three numbers:
    - The number of successful outcomes for having four winning numbers exactly
    - The number of successful outcomes for having five winning numbers exactly
    - The number of successful outcomes for having six winning numbers exactly